# Questioning ADHD and  Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [2]:
!pip install -q -U langchain openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.8.6 requires langchain<=0.0.266,>=0.0.262, but you have langchain 0.0.283 which is incompatible.


In [3]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [4]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [5]:
!pip install -q -U wikipedia

In [6]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

adhd_wikipedia_docs = WikipediaLoader(
    query="Attention deficit hyperactivity disorder", 
    load_max_docs=1, # YOUR CODE HERE, 
    doc_content_chars_max=1_000_000### YOUR CODE HERE
    ).load()

adhd_csv_docs = CSVLoader(
    file_path='adhd-twitter.csv',### YOUR CODE HERE, 
    source_column='url'### YOUR CODE HERE
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =500, ### YOUR CODE HERE,
    chunk_overlap =0, ### YOUR CODE HERE,
    length_function =len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators =["\n==","\n"," "], ### YOUR CODE HERE # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000, ### YOUR CODE HERE,
    chunk_overlap =50, ### YOUR CODE HERE,
    length_function =len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators =["\n", " "] ### YOUR CODE HERE # keep paragraphs, then sentences
)

chunked_adhd_wikipedia_docs =wikipedia_text_splitter.transform_documents(adhd_wikipedia_docs) ### YOUR CODE HERE
chunked_adhd_csv_docs =csv_text_splitter.transform_documents(adhd_csv_docs) ### YOUR CODE HERE

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [9]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [10]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store =LocalFileStore("./shared_cache/")### YOUR CODE HERE

core_embeddings_model =OpenAIEmbeddings()#### YOUR CODE HERE

#core_embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key) ### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(
core_embeddings_model, store, namespace=core_embeddings_model.model
)

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [19]:
adhd_csv_faiss_retriever =FAISS.from_documents(chunked_adhd_csv_docs,embedder).as_retriever() ### YOUR CODE HERE


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [14]:
# set up BM25 retriever / bag of words
adhd_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_adhd_wikipedia_docs
    ### YOUR CODE HERE
)
adhd_wikipedia_bm25_retriever.k = 1 # return 1 document

# set up FAISS vector store
adhd_wikipedia_faiss_store = FAISS.from_documents(
    chunked_adhd_wikipedia_docs,
    embedder
)
adhd_wikipedia_faiss_retriever = adhd_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
adhd_ensemble_retriever = EnsembleRetriever(
    retrievers=[adhd_wikipedia_bm25_retriever, adhd_wikipedia_faiss_retriever],
    weights= [0.25,0.75]### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [15]:
from langchain.agents.agent_toolkits import create_retriever_tool

adhd_wikipedia_retrieval_tool = create_retriever_tool(
    adhd_ensemble_retriever,
    "wikipedia",
    "searches and returns documents regarding the ADHD condition"
)

adhd_csv_retrieval_tool = create_retriever_tool(
    adhd_csv_faiss_retriever,
    "PublicReviews",
    "Searches and returns documents regarding patient personal experience of the adhd"
)

adhd_retriever_tools = [adhd_wikipedia_retrieval_tool, adhd_csv_retrieval_tool]

In [23]:

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
system_template = """
Use the following pieces of context to answer the user's question.
Please respond as if you were a a researcher on patient's experience. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
You can make inferences based on the context as long as it still faithfully represents the feedback.
Example of your response should be:
```
The answer is foo
```
Begin!
----------------
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
prompt = ChatPromptTemplate(messages=messages)
chain_type_kwargs = {"prompt": prompt}

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [69]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

adhd_retriever_agent_executor = create_conversational_retrieval_agent(llm, adhd_retriever_tools, verbose=True, chain_type_kwargs = {"prompt": prompt})
adhd_retriever_agent_executor = create_conversational_retrieval_agent(llm, adhd_retriever_tools, verbose=True)

In [70]:
res=adhd_retriever_agent_executor({"input" : "experience for people with adhd and school?"})




> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `ADHD and school`


[Document(page_content='\ufeff"fullname": ADHD posting🏳️\u200d⚧️ ( School :c )\nid: 1698395060751192474\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 2\nquotes: 0\nreplies: 2\nretweets: 0\ntext: I am awake\ntimestamp: 2023-09-03T17:58:00.000Z\nurl: https://twitter.com/PaisensADHD/status/1698395060751192474\nusername: @PaisensADHD\nverified: false', metadata={'source': 'https://twitter.com/PaisensADHD/status/1698395060751192474', 'row': 805}), Document(page_content='\ufeff"fullname": Frif 🟪\nid: 1698433179252560372\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 20\nquotes: 0\nreplies: 1\nretweets: 2\ntext: If you perform extremely well on major grades like tests and projects, but daily work and especially homework brings your grades down, you probably have ADHD.\n\nIf you\'re able to, talk to a doctor about it. Living without some kind of treatment for that is like hard mode.\ntime

In [ ]:
res

In [62]:
res=adhd_retriever_agent_executor.invoke({"input" : "experience for people with adhd and school?"})




> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `ADHD school experience`


[Document(page_content='\ufeff"fullname": unfettered by neuronormativity\nid: 1698429799126225390\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 0\nquotes: 0\nreplies: 2\nretweets: 0\ntext: h/t at Lauren Ancona, put me on the path to understanding what happened to me by sharing her story of ADHD dx +subsequent years of medication to be able to maintain in an office/work situation.\nI resisted mood medication because I was a therapist myself.+\ntimestamp: 2023-09-03T20:16:00.000Z\nurl: https://twitter.com/cyn_mccrackan/status/1698429799126225390\nusername: @cyn_mccrackan\nverified: false', metadata={'source': 'https://twitter.com/cyn_mccrackan/status/1698429799126225390', 'row': 326}), Document(page_content='\ufeff"fullname": ADHD posting🏳️\u200d⚧️ ( School :c )\nid: 1698426129668002006\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 1\nquotes: 0\nreplies: 0\nretweets: 0\ntext

In [71]:
res['output']

'Here are some personal experiences shared by individuals with ADHD and school:\n\n1. User @PaisensADHD tweeted, "I am awake." You can find more information [here](https://twitter.com/PaisensADHD/status/1698395060751192474).\n\n2. User @Frifuloid tweeted, "If you perform extremely well on major grades like tests and projects, but daily work and especially homework brings your grades down, you probably have ADHD. If you\'re able to, talk to a doctor about it. Living without some kind of treatment for that is like hard mode." You can find more information [here](https://twitter.com/Frifuloid/status/1698433179252560372).\n\n3. User @PaisensADHD tweeted, "It was 10:50ish when I posted it." You can find more information [here](https://twitter.com/PaisensADHD/status/1698408087940677886).\n\n4. User @PaisensADHD tweeted, "I\'d have to disagree." You can find more information [here](https://twitter.com/PaisensADHD/status/1698427733519577111).\n\nPlease note that these are personal experiences 

In [72]:
res['output']

'Here are some personal experiences shared by individuals with ADHD and school:\n\n1. User @PaisensADHD tweeted, "I am awake." You can find more information [here](https://twitter.com/PaisensADHD/status/1698395060751192474).\n\n2. User @Frifuloid tweeted, "If you perform extremely well on major grades like tests and projects, but daily work and especially homework brings your grades down, you probably have ADHD. If you\'re able to, talk to a doctor about it. Living without some kind of treatment for that is like hard mode." You can find more information [here](https://twitter.com/Frifuloid/status/1698433179252560372).\n\n3. User @PaisensADHD tweeted, "It was 10:50ish when I posted it." You can find more information [here](https://twitter.com/PaisensADHD/status/1698408087940677886).\n\n4. User @PaisensADHD tweeted, "I\'d have to disagree." You can find more information [here](https://twitter.com/PaisensADHD/status/1698427733519577111).\n\nPlease note that these are personal experiences 

In [105]:
res.keys()

dict_keys(['input', 'output'])

In [106]:
res

{'input': 'What is it like to be a women and has depression?',
 'output': "It is important to note that everyone's experience with depression is unique, but some common symptoms of depression in women include pervasive low mood, loss of interest or pleasure, fatigue, changes in appetite and sleep, feelings of worthlessness or guilt, difficulty concentrating, physical symptoms, irritability or restlessness, and thoughts of death or suicide. Women with depression may also experience low self-esteem, changes in sleeping and eating habits, and may be more likely to experience postpartum depression after giving birth. It is recommended to seek professional help for an accurate diagnosis and appropriate treatment if experiencing these symptoms."}

In [73]:
type(res['intermediate_steps'][0][1])

list

In [74]:
adhd_retriever_agent_executor({"input" : "What is the experience of an adhd teenager in school?"})



> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `ADHD teenager in school`


[Document(page_content='\ufeff"fullname": unfettered by neuronormativity\nid: 1698429799126225390\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 0\nquotes: 0\nreplies: 2\nretweets: 0\ntext: h/t at Lauren Ancona, put me on the path to understanding what happened to me by sharing her story of ADHD dx +subsequent years of medication to be able to maintain in an office/work situation.\nI resisted mood medication because I was a therapist myself.+\ntimestamp: 2023-09-03T20:16:00.000Z\nurl: https://twitter.com/cyn_mccrackan/status/1698429799126225390\nusername: @cyn_mccrackan\nverified: false', metadata={'source': 'https://twitter.com/cyn_mccrackan/status/1698429799126225390', 'row': 326}), Document(page_content='\ufeff"fullname": my therapist say i got ADHD 😔\nid: 1698423799920676972\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 3\nquotes: 0\nreplies: 0\nretweets: 0\ntext: U st

{'input': 'What is the experience of an adhd teenager in school?',
 'chat_history': [AIMessage(content='Here are some personal experiences shared by individuals with ADHD and school:\n\n1. User @PaisensADHD tweeted, "I am awake." You can find more information [here](https://twitter.com/PaisensADHD/status/1698395060751192474).\n\n2. User @Frifuloid tweeted, "If you perform extremely well on major grades like tests and projects, but daily work and especially homework brings your grades down, you probably have ADHD. If you\'re able to, talk to a doctor about it. Living without some kind of treatment for that is like hard mode." You can find more information [here](https://twitter.com/Frifuloid/status/1698433179252560372).\n\n3. User @PaisensADHD tweeted, "It was 10:50ish when I posted it." You can find more information [here](https://twitter.com/PaisensADHD/status/1698408087940677886).\n\n4. User @PaisensADHD tweeted, "I\'d have to disagree." You can find more information [here](https://t

In [18]:
adhd_retriever_agent_executor({"input" : "What does patients say about being female and has adhd?"})



> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `female with ADHD`


[Document(page_content='\ufeff"fullname": lizzy\nid: 1698435620023017861\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 1\nquotes: 0\nreplies: 0\nretweets: 0\ntext: there’s like different types of adhd like type?? and it will definitely be helpful to look up symptoms in WOMEN bc it changes a lot from guys, a big thing for me is off my meds i’m annoying?? like i’ve always like had that made apparent to me but def look at the symptoms\ntimestamp: 2023-09-03T20:39:00.000Z\nurl: https://twitter.com/lamictalspo/status/1698435620023017861\nusername: @lamictalspo\nverified: false', metadata={'source': 'https://twitter.com/lamictalspo/status/1698435620023017861', 'row': 249}), Document(page_content='\ufeff"fullname": Alli Covington ❤️❤️❤️\nid: 1698396781539897828\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 0\nquotes: 0\nreplies: 0\nretweets: 0\ntext: I feel this. I was diagnosed in the

{'input': 'What does patients say about being female and has adhd?',
 'chat_history': [AIMessage(content='The official symptoms of ADHD (Attention Deficit Hyperactivity Disorder) include excessive amounts of inattention, hyperactivity, and impulsivity that are pervasive, impairing in multiple contexts, and otherwise age-inappropriate. These symptoms arise from executive dysfunction, and emotional dysregulation is often considered a core symptom. In children, problems paying attention may result in poor school performance. ADHD is associated with other neurodevelopmental and mental disorders as well as some non-psychiatric disorders, which can cause additional impairment. It affects about 5-7% of children and 1-2% of adults. The condition can be difficult to distinguish from other conditions and high levels of activity within the range of normal behavior. ADHD has a negative impact on patient health-related quality of life and may increase the risk of other psychiatric conditions such a

In [21]:
res = adhd_retriever_agent_executor({"input" : "what is adhd?"})
res



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `ADHD`


[Document(page_content='Attention deficit hyperactivity disorder (ADHD) is a neurodevelopmental disorder characterised by excessive amounts of inattention, hyperactivity, and impulsivity that are pervasive, impairing in multiple contexts, and otherwise age-inappropriate.ADHD symptoms arise from executive dysfunction, and emotional dysregulation is often considered a core symptom. In children, problems paying attention may result in poor school performance. ADHD is associated with other neurodevelopmental and mental', metadata={'title': 'Attention deficit hyperactivity disorder', 'summary': 'Attention deficit hyperactivity disorder (ADHD) is a neurodevelopmental disorder characterised by excessive amounts of inattention, hyperactivity, and impulsivity that are pervasive, impairing in multiple contexts, and otherwise age-inappropriate.ADHD symptoms arise from executive dysfunction, and emotional dysregulation is

{'input': 'what is adhd?',
 'chat_history': [AIMessage(content='Attention deficit hyperactivity disorder (ADHD) is a neurodevelopmental disorder characterized by excessive amounts of inattention, hyperactivity, and impulsivity that are pervasive, impairing in multiple contexts, and otherwise age-inappropriate. ADHD symptoms arise from executive dysfunction, and emotional dysregulation is often considered a core symptom. In children, problems paying attention may result in poor school performance. ADHD is associated with other neurodevelopmental and mental disorders as well as some non-psychiatric disorders, which can cause additional impairment, especially in modern society.\n\nThe precise causes of ADHD are unknown in the majority of cases, but genetic factors play an important role. ADHD tends to run in families and has a heritability rate of 74%. Environmental factors such as toxins and infections during pregnancy, as well as brain damage, may also contribute to the development of A

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [75]:


depression_wikipedia_docs = WikipediaLoader(
    query="Major depressive disorder", 
    load_max_docs=1,  
    doc_content_chars_max=1_000_000
    ).load()

depression_csv_docs = CSVLoader(
    file_path='Mental-Health-Twitter 3.csv',
    source_column='post_id'
    ).load()

In [80]:
chunked_depression_wikipedia_docs =wikipedia_text_splitter.transform_documents(depression_wikipedia_docs) 
chunked_depression_csv_docs = csv_text_splitter.transform_documents(depression_csv_docs)

In [81]:
# public review Opp retriever
depression_csv_faiss_retriever = FAISS.from_documents(chunked_depression_csv_docs,embedder).as_retriever() 
# set up BM25 retriever
depression_wikipedia_bm25_retriever = BM25Retriever.from_documents(chunked_depression_wikipedia_docs)
depression_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
depression_wikipedia_faiss_store = FAISS.from_documents(chunked_depression_wikipedia_docs, embedder)
depression_wikipedia_faiss_retriever = depression_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
depression_ensemble_retriever = EnsembleRetriever(
    retrievers=[depression_wikipedia_bm25_retriever,depression_wikipedia_faiss_retriever], weight=[0.25,0.75]
 ) 

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [82]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: Twitter user talk about their depression.
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the depressive mood disorder including the scientific information about this depression.
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])


In [83]:
depression_multisource_chain = {
    "source1": (lambda x: x["question"]) | depression_ensemble_retriever,
    "source2": (lambda x: x["question"]) | depression_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [86]:
depression_multisource_chain.invoke({"question" : "Is depression very common?"})

AIMessage(content='Yes, depression is very common. According to the information from Source 1, major depressive disorder affected approximately 163 million people in 2017, which is about 2% of the global population. The percentage of people who are affected at one point in their life varies from 7% in Japan to 21% in France. In the United States, 8.4% of adults (21 million individuals) have at least one episode within a year-long period.', additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [87]:
from langchain.agents import Tool

## barbie_retriever_agent_executor({"input" : "Did people like Barbie"})
## oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

def query_depression(input): # because we use multi-query, 
    return depression_multisource_chain.invoke({"question" : input})

tools = [  ## this is actually 4 soures, but this pattern gives structure, the description helps it to decide which tool to use
    Tool(
        name = "AdhdInfo",#
        func=adhd_retriever_agent_executor.invoke,
        description="useful for when you need to answer questions about Adhd. Input should be a fully formed question."
    ),
    Tool(
        name = "DepressionInfo", 
        func=query_depression, 
        description="useful for when you need to answer questions about Depression. Input should be a fully formed question."
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [88]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [89]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [91]:
adhdDepression_agent = ZeroShotAgent(
    llm_chain=llm_chain, 
    tools=tools, 
    verbose=True)

adhdDepression_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=adhdDepression_agent, 
    tools=tools, 
    verbose=True)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [103]:
res=adhdDepression_agent_chain.invoke({"input" : "What is it like to be a women and has depression?"})



> Entering new AgentExecutor chain...
Thought: I need to gather information about depression in women.
Action: DepressionInfo
Action Input: "What are the symptoms of depression in women?"
Observation: content="The symptoms of depression in women can vary, but some common symptoms include:\n\n1. Pervasive low mood: Feeling sad, empty, or hopeless most of the time.\n2. Loss of interest or pleasure: Losing interest in activities or hobbies that were once enjoyable.\n3. Fatigue or low energy: Feeling tired and lacking energy, even after getting enough rest.\n4. Changes in appetite: Significant weight loss or gain, or changes in appetite.\n5. Sleep disturbances: Difficulty falling asleep, staying asleep, or sleeping too much.\n6. Feelings of worthlessness or guilt: Feeling excessive guilt or a sense of worthlessness.\n7. Difficulty concentrating or making decisions: Trouble focusing, making decisions, or remembering things.\n8. Physical symptoms: Experiencing physical symptoms such as hea

In [104]:
type(res)


dict

In [93]:
adhdDepression_agent_chain.run({"input" : "What did people hate about having adhd?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people dislike about having ADHD.
Action: AdhdInfo
Action Input: "What do people dislike about having ADHD?"

> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `dislike having ADHD`


[Document(page_content='\ufeff"fullname": unfettered by neuronormativity\nid: 1698429799126225390\nimages/0: \nisQuote: false\nisRetweet: false\nlikes: 0\nquotes: 0\nreplies: 2\nretweets: 0\ntext: h/t at Lauren Ancona, put me on the path to understanding what happened to me by sharing her story of ADHD dx +subsequent years of medication to be able to maintain in an office/work situation.\nI resisted mood medication because I was a therapist myself.+\ntimestamp: 2023-09-03T20:16:00.000Z\nurl: https://twitter.com/cyn_mccrackan/status/1698429799126225390\nusername: @cyn_mccrackan\nverified: false', metadata={'source': 'https://twitter.com/cyn_mccrackan/status/1698429799126225390', 'row': 326}), Document(page_content=

'Some common dislikes about having ADHD include the need for medication, challenges with impulsivity and hyperactivity, memory issues, and the language used to describe the condition.'

In [96]:
#adhdDepression_agent_chain.run({"input" : "what is it like to have both adhd and depression?"})

In [97]:
res=adhdDepression_agent_chain.run({"input" : "what people are say about having depression and adhd at the same time?"})



> Entering new AgentExecutor chain...
Thought: I need to gather information about people's experiences with having depression and ADHD simultaneously.
Action: DepressionInfo
Action Input: "What do people say about having depression and ADHD at the same time?"
Observation: content='People say that having depression and ADHD at the same time can complicate the diagnosis and treatment of both conditions. They mention that around a third of individuals diagnosed with ADHD develop comorbid depression. Additionally, they highlight that depression may also coexist with ADHD, making it challenging to manage both conditions effectively.' additional_kwargs={} example=False
Thought:I need to gather more information about the complications of having depression and ADHD at the same time.
Action: AdhdInfo
Action Input: "What are the complications of having depression and ADHD at the same time?"

> Entering new AgentExecutor chain...
Having depression and ADHD at the same time can lead to several c

In [107]:
res=adhdDepression_agent_chain({"input" : "what people are say about having depression and adhd at the same time?"})



> Entering new AgentExecutor chain...
Thought: I need to gather information about people's experiences with having depression and ADHD simultaneously.
Action: DepressionInfo
Action Input: "What are people's experiences with having depression and ADHD at the same time?"
Observation: content="People's experiences with having depression and ADHD at the same time can vary, but there are some common themes. Many individuals report feeling overwhelmed and struggle with managing their symptoms of both conditions. They may experience difficulties with concentration, motivation, and organization, which can exacerbate feelings of low mood and hopelessness. The co-occurrence of depression and ADHD can also complicate the diagnosis and treatment process, as symptoms may overlap and be difficult to distinguish. Additionally, individuals with both conditions may be at a higher risk for substance use disorders and may require a comprehensive treatment approach that addresses both depression and ADH

In [108]:
res

{'input': 'what people are say about having depression and adhd at the same time?',
 'output': "People's experiences with having depression and ADHD at the same time can vary, but there are some common themes. Many individuals report feeling overwhelmed and struggle with managing their symptoms of both conditions. They may experience difficulties with concentration, motivation, and organization, which can exacerbate feelings of low mood and hopelessness. The co-occurrence of depression and ADHD can also complicate the diagnosis and treatment process, as symptoms may overlap and be difficult to distinguish. Additionally, individuals with both conditions may be at a higher risk for substance use disorders and may require a comprehensive treatment approach that addresses both depression and ADHD. The prevalence of comorbid depression and ADHD is around one-third of individuals diagnosed with ADHD."}

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: